In [1]:
import deepdish as dd
import os
import numpy as np

In [2]:
python = '/Users/gravity/opt/anaconda3/envs/gwforge_18Oct24/bin/python'

In [17]:
#os.system(f'{python} gen_WF_calc_SNR.py \
            --param-file /Users/gravity/ACADEMIC/Projects/Cardiff_University/gwforge/test_pop_day.h5 \
            --approximant IMRPhenomPv2 \
            --detectors-and-psds \
                L1:aLIGOZeroDetHighPower \
                H1:aLIGOZeroDetHighPower \
                V1:AdvVirgo \
            --is-asd \
            --set-name 31_days_prec_z_10_')

File found


Checking if this is a file path..


Calculating SNR for L1
Calculating SNR for H1
Calculating SNR for V1
Writing results to file
Done!


0

In [38]:
test_data = dd.io.load('../output_data/1_day_prec_z_10_SNR_data.h5')['Optimal_SNR']

In [39]:
test_data.keys()

Index(['inclination', 'distance', 'mass1', 'mass2', 'coa_phase', 'f_ref',
       'spin1x', 'spin1y', 'spin1z', 'spin2x', 'spin2y', 'spin2z',
       'approximant', 'f_lower', 'delta_f', 'f_final', 'dec', 'trigger_time',
       'polarization', 'ra', 'SNR_L1', 'SNR_H1', 'SNR_V1', 'SNR_network_sq',
       'SNR_network'],
      dtype='object')

In [40]:
SNRs = {}
detected_SNRs = {}
SNR_threshold = 10
for det in ['L1', 'H1', 'V1', 'network', 'network_sq']:
    if det == 'network_sq':
        SNR_threshold = SNR_threshold**2
    SNRs[det] = np.array(test_data['SNR_'+det])
    detected_SNRs[det] = SNRs[det][np.where(SNRs[det] >= SNR_threshold)]

In [41]:
netw_sq = SNRs['H1']**2 + SNRs['L1']**2 + SNRs['V1']**2
for i in range(9):
    print(SNRs['H1'][i]**2 + SNRs['L1'][i]**2 + SNRs['V1'][i]**2, netw_sq[i], SNRs['network_sq'][i])

0.8712575053591547 0.8712575053591547 0.8712575053591547
0.5668601385907314 0.5668601385907314 0.5668601385907314
47.83723459159324 47.83723459159324 47.83723459159324
0.6815772415593295 0.6815772415593295 0.6815772415593295
1.0493144572807556 1.0493144572807556 1.0493144572807556
0.22184313321502608 0.22184313321502608 0.22184313321502608
1.1698793222009567 1.1698793222009567 1.1698793222009567
0.7022796639920941 0.7022796639920941 0.7022796639920941
0.25419825729008233 0.25419825729008233 0.25419825729008233


In [42]:
detected_SNRs

{'L1': array([], dtype=float64),
 'H1': array([], dtype=float64),
 'V1': array([10.93770602]),
 'network': array([12.49747802, 12.54552827]),
 'network_sq': array([156.18695676, 157.39027967])}